In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWhmtB-P7ib1p73aIDbQ7YgW8gv9Y12eXbYU72jS691FGROKvtNVhIg



Successfully saved authorization token.


### Imports and Constants
##### test areas, data sources, composite RGB image

In [21]:
import ee
import geemap

Map = geemap.Map()

area1 = ee.Geometry.Polygon(
        [[[25.75, -10.75],
          [25.75, -10.8],
          [25.9, -10.8],
          [25.9, -10.75]]], None, False)
area2 = ee.Geometry.Polygon(
        [[[25.8, -10.6],
          [25.8, -10.7],
          [25.95, -10.7],
          [25.95, -10.6]]], None, False)
area3 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]], None, False)
area4 = ee.Geometry.Polygon(
        [[[25.95, -10.6],
          [25.95, -10.7],
          [26.05, -10.7],
          [26.05, -10.6]]], None, False)
noMine = ee.Geometry.Polygon(
        [[[26.05, -10.7],
          [26.05, -10.8],
          [26.15, -10.8],
          [26.15, -10.7]]], None, False)
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')
modis = ee.ImageCollection("MODIS/006/MOD13Q1") # vegetation indices
modis2 = ee.ImageCollection('MODIS/006/MCD43A4')
rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
comp2020 = s2 \
            .filter(ee.Filter.bounds(area4)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(area4)
sar2020 = s1 \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters',10)) \
            .filter(ee.Filter.intersects('.geo', area4)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .select(['VV', 'VH']).median().clip(area4)

### Create .05x.05 Regions

In [22]:
# must be rectangular geometry
def divide_geometry(geo):
    regions = []
    coords = geo.coordinates().getInfo()[0]
    i = coords[0][0]
    while i <= coords[1][0]:
        j = coords[2][1]
        while j >= coords[0][1]:
            region = ee.Geometry.Polygon(
                [[[i, j],
                  [i, j-.05],
                  [i+.05, j-.05],
                  [i+.05, j]]])
            regions.append((region))
            j = j-.05
        i = i+.05
    return regions

In [23]:
regions = divide_geometry(area4)
print('Number of Regions Created: ' + str(len(regions)))

Number of Regions Created: 6


### Create Indices: NDVI, NDMI, NIR/SWIR2

##### NDVI: Normalized Difference Vegetation Index
##### Using Modis NDVI band, values range from -2000 to 10000

In [5]:
def create_ndvi_layers_modis(start, end, geometry):
    images = []
    ndviStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviStart, start))
    ndviEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviEnd, end))
    return images

##### NDMI: Normalized Difference Moisture Index
##### Using NIR (band 2) and SWIR1 (band 6)

In [6]:
def create_ndmi_layers_modis(start, end, geometry):
    images = []
    ndmiStart = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiStart2 = ndmiStart.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiStart2, start))
    ndmiEnd = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiEnd2 = ndmiEnd.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiEnd2, end))
    return images

##### NIR/SWIR2 - using normalized difference
##### NIR (band 2) and SWIR2 (band 7)

In [7]:
def create_nirSwir2_layers_modis(start, end, geometry):
    images = []
    nirSwir2Start = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2Start2 = nirSwir2Start.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2Start2, start))
    nirSwir2End = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2End2 = nirSwir2End.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2End2, end))
    return images

##### Functions to generate an average index value for a given region and year, for each desired index

In [8]:
def extract_ndvi_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDVI': stats.get('NDVI')
    }
    
    return ee.Feature(None, properties)

In [9]:
def extract_ndmi_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDMI': stats.get('NDMI')
    }
    
    return ee.Feature(None, properties)

In [10]:
def extract_nirSwir2_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'nirSwir2': stats.get('nirSwir2')
    }
    
    return ee.Feature(None, properties)

##### Functions to determine if the loss/gain of each index for a given region over given years is significant enough to be a possible mining location

In [11]:
def ndvi_requirement_modis(df):
    status = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1] < 4000:
        status = 1
    elif df.iat[0,1]-df.iat[1,1] > 1000: 
        status = 1
    else: 
        status = status
    return status

In [38]:
def ndmi_requirement_modis(df):
    statusNDMI = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1] < 0: 
        statusNDMI = 1
    elif df.iat[0,1]-df.iat[1,1] > .1: 
        statusNDMI = 1
    else: 
        statusNDMI = statusNDMI
    return statusNDMI

In [13]:
def nirSwir2_requirement_modis(df):
    statusNS = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1] < .4: 
        statusNS = 1
    elif df.iat[0,1]-df.iat[1,1] > .25: 
        statusNS = 1
    else: 
        statusNS = statusNS
    return statusNS

##### Function to determine what regions within an area may be possible mine locations based on all three indices, and adds these regions to a map as composite images 

In [36]:
def mining_modis(regions):
    Map.centerObject(area4, 12)
    Map.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' pass'
        
        ## NDVI
        imagesNDVI = ee.ImageCollection(create_ndvi_layers_modis(2004, 2020, region))
        dataNDVI = ee.FeatureCollection(imagesNDVI.map(extract_ndvi_modis))
        dfNDVI = geemap.ee_to_pandas(dataNDVI)

        status = ndvi_requirement_modis(dfNDVI)
        
        if status == 1:
            
            ## NDMI
            imagesNDMI = ee.ImageCollection(create_ndmi_layers_modis(2004, 2020, region))
            dataNDMI = ee.FeatureCollection(imagesNDMI.map(extract_ndmi_modis))
            dfNDMI = geemap.ee_to_pandas(dataNDMI)
            
            statusNDMI = ndmi_requirement_modis(dfNDMI)
            
            if statusNDMI == 1:
                
                ## NIR/SWIR2
                imagesNirSwir2 = ee.ImageCollection(create_nirSwir2_layers_modis(2004, 2020, region))
                dataNirSwir2 = ee.FeatureCollection(imagesNirSwir2.map(extract_nirSwir2_modis))
                dfNirSwir2 = geemap.ee_to_pandas(dataNirSwir2)
            
                statusNS = nirSwir2_requirement_modis(dfNirSwir2)
                
                if statusNS == 1:
                    
                    image = s2 \
                        .filter(ee.Filter.bounds(region)) \
                        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                        .select('B.*') \
                        .median().clip(region)
                    Map.addLayer(image, rgbVis, layerName)
                    
                    nirG = image.normalizedDifference(['B8', 'B3'])
                    ndmi = image.normalizedDifference(['B8', 'B11'])
                    iron = image.normalizedDifference(['B4', 'B2']).rename('iron')
                    vv2020 = sar2020.select('VV')
                    bsi = image.expression('(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
                        'X': image.select('B11'), #swir1
                        'Y': image.select('B4'),  #red
                        'A': image.select('B8'), # nir
                        'B': image.select('B2'), # blue
                        }).rename('bsi')
        
                    waterStorage = vv2020.lt(-15).rename('water').And(nirG.lt(0.1).Or(ndmi.gt(.3))).selfMask()
                    connectW = waterStorage.connectedPixelCount(25)
                    waterStorage = waterStorage.updateMask(connectW.gt(8))
                    Map.addLayer(waterStorage, {'min':0, 'max':1, 'palette':['blue']}, str(num) + ' Water')
                    
                    ironSpots = iron.gt(0.4).rename('iron').selfMask()
                    connectI = ironSpots.connectedPixelCount(25)
                    ironSpots = ironSpots.updateMask(connectI.gt(8))
                    Map.addLayer(ironSpots, {'min':0, 'max':1, 'palette':['gray']}, str(num) + ' Iron')
                    
                    bareSoil = bsi.gt(0.23).rename('bsi').selfMask()
                    connectB = bareSoil.connectedPixelCount(25)
                    bareSoil = bareSoil.updateMask(connectB.gt(8))
                    Map.addLayer(bareSoil, {'min':0, 'max':1, 'palette':['red']}, str(num) + ' Mining')
                    
        num = num+1
    return Map

##### Calls the above function on the selected regions
##### Overlays predicted water storage, iron, and bsi (mines)

In [37]:
mining_modis(regions)

Map(bottom=555789.0, center=[-10.649998588349076, 26.000000000000174], controls=(WidgetControl(options=['posit…